In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as T

import os
import random

# from models import *
# from loader import Loader, RotationLoader
from utils import progress_bar
import numpy as np

import string
import glob 
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import IPython.display as ipd

# 1 Setting

## 1.1 Argument

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
best_acc = 0 # best test acc
start_epoch = 0 # start from epoch 0 OR checkpoint epoch

## 1.2 Data

In [4]:
DATASET = 'Large_Captcha'

In [5]:
all_letters = string.ascii_lowercase + string.digits + string.ascii_uppercase

mapping = {}      # key - num    & value - letter
mapping_inv = {}  # key - letter & value - num

for i, x in enumerate(all_letters):
    mapping[x] = i+1
    mapping_inv[i+1] = x

num_class = len(mapping)
num_class

62

In [6]:
class Loader_Captcha_active(Dataset):
    def __init__(self, is_train=True, transform=None, path_list=None):
        self.is_train = is_train
        self.transform = transform
        if path_list is None:
            raise Exception("not have a path_list")
        self.img_path = path_list

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        #img = cv2.imread(self.img_path[idx])
        #img = Image.fromarray(img)
        img = Image.open(self.img_path[idx]).convert('L')
        if self.transform is not None:
            img = self.transform(img)
        
        img_name = self.img_path[idx].split('/')[-1]
        label = torch.IntTensor([mapping[i] for i in img_name.split('.')[0]]) #int(self.img_path[idx].split('/')[-1])
        
        return img, label

In [7]:
transform = T.Compose([
    T.ToTensor()
])

In [8]:
import pickle
testset = []
with open('./testset.pkl', 'rb') as f:
    testset = pickle.load(f)
len(testset)

16466

In [9]:
random.shuffle(testset)
testloader  = torch.utils.data.DataLoader(testset[:1000],  batch_size=8,  shuffle=False)

## 1.2 Model

In [10]:
class Bidirectional(nn.Module):
    def __init__(self, inp, hidden, out, lstm=True):
        super(Bidirectional, self).__init__()
        if lstm:
            self.rnn = nn.LSTM(inp, hidden, bidirectional=True)
        else:
            self.rnn = nn.GRU(inp, hidden, bidirectional=True)
        self.embedding = nn.Linear(hidden*2, out)
    def forward(self, X):
        recurrent, _ = self.rnn(X)
        out = self.embedding(recurrent)     
        return out

In [11]:
class CRNN(nn.Module):
    def __init__(self, in_channels, output):
        super(CRNN, self).__init__()

        self.cnn = nn.Sequential(
                nn.Conv2d(in_channels, 256, 9, stride=1, padding=1),
                nn.ReLU(),
                nn.BatchNorm2d(256),
                nn.MaxPool2d(3, 3),
                nn.Conv2d(256, 256, (4, 3), stride=1, padding=1),
                nn.ReLU(),
                nn.BatchNorm2d(256))
        
        self.linear = nn.Linear(20992, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.rnn = Bidirectional(256, 1024, output+1)

    def forward(self, X, y=None, criterion = None): # y is target.
        out = self.cnn(X)
        N, C, w, h = out.size()
        out = out.view(N, -1, h)
        out = out.permute(0, 2, 1)
        out = self.linear(out)

        out = out.permute(1, 0, 2)
        out = self.rnn(out)
            
        if y is not None:
            T = out.size(0)
            N = out.size(1)
        
            input_lengths = torch.full(size=(N,), fill_value=T, dtype=torch.int32)
            target_lengths = torch.full(size=(N,), fill_value=5, dtype=torch.int32)
        
            loss = criterion(out, y, input_lengths, target_lengths)
            
            return out, loss
        
        return out, None
    
    def _ConvLayer(self, inp, out, kernel, stride, padding, bn=False):
        if bn:
            conv = [
                nn.Conv2d(inp, out, kernel, stride=stride, padding=padding),
                nn.ReLU(),
                nn.BatchNorm2d(out)
            ]
        else:
            conv = [
                nn.Conv2d(inp, out, kernel, stride=stride, padding=padding),
                nn.ReLU()
            ]
        return nn.Sequential(*conv)

In [12]:
model = CRNN(in_channels=1, output=num_class)
model = model.to(device)
# print(net)

In [13]:
if device == 'cuda':
    #net = torch.nn.DataParallel(net) # 문현안쓸때만쓰기... 나는 0
    cudnn.benchmark = True

In [14]:
# criterion = nn.CTCLoss() #nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-4) # optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
# # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 60, 90])

# 2 Training

In [15]:
def predict(outputs):
    result = []
    for i in range(len(outputs)):
        pred = []
        then = 0
        for x in outputs[i]:
            if then != x and x > 0 :
                pred.append(x)
                if len(pred) == 5:
                    break
            then = x
        if len(pred) < 5:
            for i in range(5-len(pred)):
                pred.append(0)
        result.append(pred)
    result = torch.LongTensor(result).cuda()
    return result

In [16]:
def train(model, criterion, optimizer, epoch, trainloader):
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    
    
    tk = tqdm(trainloader, total=len(trainloader))
    #for batch_idx, (inputs, targets) in enumerate(trainloader):
    for inputs, targets in tk:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        
        out, loss = model(inputs, targets, criterion=criterion)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        predicted = predict(out.permute(1, 2, 0).max(1)[1])
        total += targets.size(0) #* 5
        for i in range(len(predicted)):
            correct += torch.equal(predicted[i], targets[i])
#         correct += predicted.eq(targets).sum().item()
        
        tk.set_postfix({'Train - Loss' : loss.item(), '& ACC':100.*correct/total})
    
    return 100.*correct/total, train_loss/total

# 3 Test

In [17]:
def test(model, criterion, epoch, cycle):
    global best_acc, DATASET
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        tk = tqdm(testloader, total=len(testloader))
#         for batch_idx, (inputs, targets) in enumerate(testloader):
        for inputs, targets in tk:
            inputs, targets = inputs.to(device), targets.to(device)
            out, loss = model(inputs, targets, criterion=criterion)
            
            test_loss += loss.item()
            predicted = predict(out.permute(1, 2, 0).max(1)[1])
            total += targets.size(0) #* 5
            
#             return predicted, targets
        
            for i in range(len(predicted)):
                correct += torch.equal(predicted[i], targets[i])
#             correct += predicted.eq(targets).sum().item()/5

            tk.set_postfix({'Test - Loss' : loss.item(), '& ACC':100.*correct/total})

    # Save checkpoint.
    acc = 100.*correct/total
#     if acc > best_acc:
#         print('Saving..')
#         state = {
#             'model': model.state_dict(),
#             'acc': acc,
#             'epoch': epoch,
#         }
#         if not os.path.isdir('checkpoint'):
#             os.mkdir('checkpoint')
#         # save rotation weights
#         torch.save(state, './checkpoint/main_'+DATASET+'_'+str(cycle)+'.pth')
#         best_acc = acc
    
    return acc, test_loss/total

# 4 Run

In [18]:
slice_num =2500

In [19]:
# confidence sampling (pseudo labeling)
## return 1k samples w/ lowest top1 score
def get_plabels2(model, samples, cycle):
    sub_dataset = Loader_Captcha_active(is_train=False,  transform=transform, path_list=samples)
    ploader = torch.utils.data.DataLoader(sub_dataset, batch_size=1, shuffle=False)

    top1_scores = []
    model.eval()
    with torch.no_grad():
        tk = tqdm(ploader, total=len(ploader))
        for inputs, targets in tk:
            inputs, targets = inputs.to(device), targets.to(device)
            out, loss = model(inputs, targets, criterion=nn.CTCLoss())
            predicted = predict(out.permute(1, 2, 0).max(1)[1])
            
            # save top1 confidence score 
            outputs = F.normalize(out.permute(1, 2, 0), dim=1)
            probs = F.softmax(out.permute(1, 2, 0), dim=1)

            # our ouput num is 5 >> mean >> max  (가장 높은 값을 뽑아 낮은 data로 정렬) 
            top1_scores.append(np.mean(probs[0][predicted.cpu()].tolist()[0], axis=0).max())
    idx = np.argsort(top1_scores)
#     return idx
    samples = np.array(samples)
    return samples[idx[:slice_num]]

In [20]:
def main():
    labeled=[]
    CYCLES = len(glob.glob('./loss_'+DATASET+'/batch_*.txt'))

    history_train = {'acc':[[] for i in range(CYCLES)], 'loss':[[] for i in range(CYCLES)]}
    history_test = {'acc':[[] for i in range(CYCLES)], 'loss':[[] for i in range(CYCLES)]}
    
    model = CRNN(in_channels=1, output=num_class)
    model = model.to(device)
    criterion = nn.CTCLoss() #nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3) # optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

    for cycle in range(CYCLES):
    #     ipd.clear_output(wait=True)
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[160])

        best_acc = 0
    #     print('Cycle ', cycle)

        # open mini batch (sorted low->high)
        with open('./loss_'+DATASET+'/batch_' + str(cycle) + '.txt', 'r') as f:
            samples = f.readlines()[:-2]
        samples = [s_path[:-1] for s_path in samples] #  \n 지우기

        if cycle > 0:
            print('>> Getting previous checkpoint')
    #         checkpoint = torch.load(f'./checkpoint/main_{DATASET}_{cycle-1}.pth')
    #         model.load_state_dict(checkpoint['model'])

            # sampling
            sample_sort = get_plabels2(model, samples, cycle)
        else:
            # first iteration: sample 1k at even intervals
            samples = np.array(samples)
            random.shuffle(samples)
            sample_sort = samples[:slice_num]
        # add 1k samples to labeled set
        labeled.extend(sample_sort)
    #     print(labeled[-1])
        print(f'>> Labeled length: {len(labeled)}')
        activeset = Loader_Captcha_active(is_train=False,  transform=transform, path_list=labeled)
        trainloader = torch.utils.data.DataLoader(activeset, batch_size=10, shuffle=True)

        model = CRNN(in_channels=1, output=num_class)
        model = model.to(device)
        criterion = nn.CTCLoss() #nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=1e-3) # optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
        # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 60, 90])
        # print(net)

    #     for m in model.modules():
    #         if isinstance(m, nn.Conv2d):
    #             m.reset_parameters()
    #         elif isinstance(m, nn.BatchNorm2d):
    #             m.reset_parameters()
    #         elif isinstance(m, nn.Linear):
    #             m.reset_parameters()
    #         elif isinstance(m, nn.BatchNorm1d):
    #             m.reset_parameters()
    #         elif isinstance(m, nn.LSTM):
    #             m.reset_parameters()

        for epoch in range(50):
            ipd.clear_output(wait=True)
            print('Cycle ', cycle)
            print(f'>> Labeled length: {len(labeled)}')

            acc, loss = train(model, criterion, optimizer, epoch, trainloader)
            history_train['acc'][cycle].append(acc)
            history_train['loss'][cycle].append(loss)

            acc, loss = test(model, criterion, epoch, cycle)
            history_test['acc'][cycle].append(acc)
            history_test['loss'][cycle].append(loss)

    #         train(model, criterion, optimizer, epoch, trainloader)
    #         test(model, criterion, epoch, cycle)
            scheduler.step()
        with open(f'./main_best'+DATASET+'.txt', 'a') as f:
            f.write(str(cycle) + ' ' + str(best_acc)+'\n')

In [ ]:
main()

Cycle  1
>> Labeled length: 5000

Epoch: 49


100%|█████████████| 125/125 [00:06<00:00, 20.14it/s, Test - Loss=1.9, & ACC=2.6]


>> Getting previous checkpoint


 41%|███████████████▊                       | 2030/4998 [00:29<00:43, 68.65it/s]